### Méthode 2:

Avec cette méthode, on fera du feature extraction de BERT, c'est à dire utiliser son output pour faire de la classification avec un autre model, en l'occurrence un `LogisticRegressionCV` classifier. 

Une meilleure representation des prenoms sera extraite avec les embedings de bert, même si la notion de contexte n'est pas présente dans notre probleme.



In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import transformers
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
df = pd.read_csv('dataframe_names.csv')

In [3]:
df = df.drop('Unnamed: 0',axis=1)

In [4]:
df.shape

(11627, 3)

In [5]:
df.drop_duplicates(subset ="prenom",
                     keep = False, inplace = True)

In [6]:
df.shape

(11386, 3)

In [7]:
df

,prenom,genre,fréquence
0,aaliyah,0,0.0
1,aapeli,1,0.0
2,aapo,1,0.0
3,aaren,2,0.0
4,aarne,1,0.0
...,...,...,...
11622,zvi,1,0.1
11623,zvonimir,1,0.0
11624,zvonimira,0,0.0
11625,zvonko,1,0.0


In [8]:
df[df.prenom.isna()==True]

,prenom,genre,fréquence
7856,NaN,0,0.92


In [9]:
df = df.dropna()

In [10]:
df[df.prenom.isna()==True]

,prenom,genre,fréquence


In [11]:
x_train, x_test, y_train, y_test = train_test_split(df['prenom'], df['genre'], test_size=0.3)

In [12]:

model = transformers.BertModel.from_pretrained("bert-base-uncased")
tokenizer =  transformers.BertTokenizer.from_pretrained('bert-base-uncased')

def transform(sentences):
    tokenized = sentences.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
    # calculer la taille la plus grande, pour ensuite completer avec 
    # la difference pour avoir une taille unique maximale de taille max_len
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)
    # on rajoute le padding 
    tokenized_pad = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
    input_ids = torch.tensor(tokenized_pad)
    # on detach le gradient
    with torch.no_grad(): 
        embeddings  = model(input_ids)
    return embeddings[0][:, 0, :].numpy()



In [15]:
tokens = transform(x_train)
test_tokens =transform(x_test)

In [21]:
def evaluate(algo):
    if algo.__name__== 'LogisticRegressionCV': clf = LogisticRegressionCV(cv=5,max_iter= 10000)
    elif algo.__name__== 'RandomForestClassifier': clf = RandomForestClassifier(max_depth=3)
    else: clf = algo()
    clf.fit(tokens, y_train)
    predicted = clf.predict(test_tokens)
    print(classification_report(y_test,predicted))    
    return clf

In [22]:
evaluate(LogisticRegressionCV)

              precision    recall  f1-score   support

           0       0.85      0.85      0.85      1555
           1       0.84      0.88      0.86      1745
           2       0.36      0.04      0.08       116

    accuracy                           0.84      3416
   macro avg       0.68      0.59      0.59      3416
weighted avg       0.82      0.84      0.83      3416



LogisticRegressionCV(cv=5, max_iter=10000)

In [23]:
evaluate(RandomForestClassifier)

              precision    recall  f1-score   support

           0       0.88      0.70      0.78      1555
           1       0.74      0.93      0.83      1745
           2       0.00      0.00      0.00       116

    accuracy                           0.79      3416
   macro avg       0.54      0.54      0.53      3416
weighted avg       0.78      0.79      0.78      3416



C:\Users\Akaen\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Akaen\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Akaen\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

RandomForestClassifier(max_depth=3)

Remarque:

En terme d'accuracy, la `LogisticRegression` s'en sort un peux mieux, même au niveau de la precision de classification de la classe numero 2, mais assez équivalents au niveau du rappel et du f1-score.

On obtient de meilleurs résultats qu'avec la méthode 1 (feature engineering), et ce en évitant d'extraire les features une à une et ainsi prendre le risque d'overfiter si jamais on ne fait pas attention.

Avec la troisième méthode on essaiera de fine-tune Bert pour voir si on obtient de meilleurs résultats qu'ici.